# Environment

In [ ]:
# setting the random seed for reproducibility
import random
random.seed(493)

# for manipulating dataframes
import pandas as pd
import numpy as np

# for statistical testing
from scipy import stats
from scipy.stats import mannwhitneyu

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

# for natural language processing: named entity recognition
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

# for working with timestamps
from datetime import datetime
from dateutil.parser import parse

# for visualizations
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set(style="whitegrid")

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

# Get Data

In [ ]:
# Read a csv file
df = pd.read_csv('../data/in/short_survey_data_with_asat_rr_csat.csv')

# Read an excel file
df = pd.read_excel('../data/in/short_survey_data_with_asat_rr_csat.xlsx')

# Read a tsv file
df = pd.read_csv('../data/in/short_survey_data_with_asat_rr_csat.tsv', sep='\t')

# Reading All Files Within a Folder

In [ ]:
# From Tarun Dey

# Show Missing Values

In [ ]:
df = pd.read_csv('../data/in/short_survey_data_with_asat_rr_csat.csv')

def show_missing(df):
    """
    Takes a dataframe and a column and returns a dataframe of
    total missing values and percent missing values by column.

            Parameters:
                    df (dataframe): A pandas dataframe

            Returns:
                    dfx (dataframe): A list of cleaned words
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_count / df.shape[0]) * 100
    dfx = pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_count': nan_count, 'nan_percentage': nan_percentage})
    return dfx

show_missing(df)

# Show Value Counts

In [ ]:
df = pd.read_csv('../data/in/short_survey_data_with_asat_rr_csat.csv')

def get_values(df, columns):
    """
    Takes a dataframe and a target column name and prints out the
    value counts of the specific column.

            Parameters:
                    df (dataframe): A pandas dataframe

            Returns:
                    null (NoneType):
    """
    for column in columns:
        print(column)
        print('=====================================')
        print(df[column].value_counts(dropna=False))
        print('\n')

def show_values(df, param):
    if param == 'all':
        get_values(df, df.columns)
    else:
        get_values(df, param) 

show_values(df, ['asat'])
show_values(df, 'all')

# Basic Clean

In [ ]:
ADDITIONAL_STOPWORDS = ['ignore', 'me']

def clean(sentence):
    """
    Takes a string and returns a list of cleaned words. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.

            Parameters:
                    sentence (str): A list of words

            Returns:
                    word_list (list): A list of cleaned words
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    sentence = (unicodedata.normalize('NFKD', sentence)
        .encode('ascii', 'ignore')
        .decode('utf-8', 'ignore')
        .lower())
    words = re.sub(r'[^\w\s]', '', sentence).split()
    word_list = [wnl.lemmatize(word) for word in words if word not in stopwords]
    return word_list

clean('The quick brown fox jumps over the lazy dog. Ignore me.')

# Get Words

In [ ]:
df = pd.read_csv('../data/in/short_survey_data_with_asat_rr_csat.csv')

def get_words(df, column):
    """
    Takes a dataframe and a column and returns a list of
    cleaned words that is returned by clean().

            Parameters:
                    df (dataframe): A pandas dataframe
                    column (series): A pandas series

            Returns:
                    word_list (list): A list of cleaned words
    """
    return clean(''.join(str(df[column].tolist())))

get_words(df, 'comment')

# Unique Words

In [ ]:
df_csat =  df.loc[df['score'] == 1]
df_dsat =  df.loc[df['score'] == 0]

In [ ]:
csat_words = get_words(df_csat, 'comment')
dsat_words = get_words(df_dsat, 'comment')
all_words = get_words(df, 'comment')

csat_freq = pd.Series(csat_words).value_counts()
dsat_freq = pd.Series(dsat_words).value_counts()
all_freq = pd.Series(all_words).value_counts()

word_counts = (pd.concat([all_freq, csat_freq, dsat_freq], axis=1, sort=True)
                .set_axis(['all', 'csat', 'dsat'], axis=1, inplace=False)
                .fillna(0)
                .apply(lambda s: s.astype(int)))

In [ ]:
# What are the most frequently occuring words?
word_counts.sort_values(by='all', ascending=False).head(10)

In [ ]:
# Are there any words that uniquely identify a dsat or csat comment?
pd.concat([word_counts[word_counts.dsat == 0].sort_values(by='csat').tail(10),
           word_counts[word_counts.csat == 0].sort_values(by='dsat').tail(10)])

# Get N-grams

In [ ]:
def get_unigrams(words):
    """
    Takes in a list of words and returns a series of
    unigrams with value counts.
    """
    return  pd.Series(words).value_counts()

def get_bigrams(words):
    """
    Takes in a list of words and returns a series of
    bigrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 2)).value_counts())[:20]

def get_trigrams(words):
    """
    Takes in a list of words and returns a series of
    trigrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 3)).value_counts())[:20]

def get_qualgrams(words):
    """
    Takes in a list of words and returns a series of
    qualgrams with value counts.
    """
    return (pd.Series(nltk.ngrams(words, 4)).value_counts())[:20]

def get_ngrams(df,column):
    """
    Takes in a dataframe with column name and generates a
    dataframe of unigrams, bigrams, trigrams, and qualgrams.
    """
    return get_bigrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'bigram','0':'count'}), \
           get_trigrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'trigram','0':'count'}), \
           get_qualgrams(get_words(df,column)).to_frame().reset_index().rename(columns={'index':'qualgram','0':'count'})

# Visualize N-grams with Bar Charts

In [ ]:
def viz_bigrams(df, column, title):
    """
    Takes in a dataframe, target column name, and specified title
    for the bar chart visualization of bigrams.
    """
    get_bigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title(title)
    plt.ylabel('Bigram')
    plt.xlabel('# Occurances')

def viz_trigrams(df, column, title):
    """
    Takes in a dataframe, target column name, and specified title
    for the bar chart visualization of trigrams.
    """
    get_trigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title(title)
    plt.ylabel('Trigram')
    plt.xlabel('# Occurances')
    
def viz_qualgrams(df, column, title):
    """
    Takes in a dataframe, target column name, and specified title
    for the bar chart visualization of qualgrams.
    """
    get_bigrams(get_words(df,column)).sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

    plt.title(title)
    plt.ylabel('Qualgram')
    plt.xlabel('# Occurances')

# Concatenating DataFrames Together

In [ ]:
df = pd.concat([df1,df2,df3])

# Merging Dataframe Together

In [ ]:
df_merged = df1.merge(df2,
                      left_on='id1',
                      right_on='id2',
                      suffixes=('_left', '_right'))

# Using iLoc to Select Rows of a DataFrame

In [ ]:
# Single selections using iloc and DataFrame

# Rows:
df.iloc[0] # first row of data frame (Aleshia Tomkiewicz) - Note a Series data type output.
df.iloc[1] # second row of data frame (Evan Zigomalas)
df.iloc[-1] # last row of data frame (Mi Richan)

# Columns:
df.iloc[:,0] # first column of data frame (first_name)
df.iloc[:,1] # second column of data frame (last_name)
df.iloc[:,-1] # last column of data frame (id)

# Multiple row and column selections using iloc and DataFrame

df.iloc[0:5] # first five rows of dataframe
df.iloc[:, 0:2] # first two columns of data frame with all rows
df.iloc[[0,3,6,24], [0,5,6]] # 1st, 4th, 7th, 25th row + 1st 6th 7th columns.
df.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of data frame (county -> phone1).

# Using loc to Select Rows of a DataFrame

In [ ]:
# Select rows with first name Ednalyn, include all columns between 'city' and 'email'
df.loc[data['first_name'] == 'Ednalyn', 'city':'email']
 
# Select rows where the email column ends with 'gmail.com', include all columns
df.loc[data['email'].str.endswith("gmail.com")]   
 
# Select rows with first_name equal to some values, all columns
df.loc[data['first_name'].isin(['Ednalyn', 'Ederlyne', 'Edelyn'])]   
       
# Select rows with first name Ednalyn and gmail email addresses
df.loc[data['email'].str.endswith("gmail.com") & (data['first_name'] == 'Ednalyn')] 
 
# select rows with id column between 100 and 200, and just return 'zip' and 'web' columns
df.loc[(data['id'] > 100) & (df['id'] <= 200), ['zip', 'web']]

# Displaying the First and Last Rows of a DataFrame

In [ ]:
df.head()
df.tail()

# Getting the Shape of a DataFrame

In [ ]:
df.shape

# Displaying the First Ten Items of a List

In [ ]:
my_list[:10]

# Displaying Columns

In [ ]:
df.columns

# Specifying Column Names in Bulk

In [ ]:
df.columns=['column_name', 'column_name', 'column_name', 'column_name', 'column_name']

# Handpicking Columns

In [ ]:
dfx = df[['column_name',
          'column_name',
          'column_name',
          'column_name',
          'column_name'
        ]]

# Renaming Columns

In [ ]:
df = df.rename(columns={'old_name':'new_name',
                        'old_name':'new_name',
                        'old_name':'new_name',
                        'old_name':'new_name',
                        'old_name':'new_name'
                        })

# Dropping Duplicates

In [ ]:
df = df.drop_duplicates(subset ="column_id", keep = False)

# Dropping Columns

In [ ]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1','score'], inplace=True)

# Setting the Index

In [ ]:
df.set_index('date_time_zone', inplace=True)

# Resetting the Index

In [ ]:
df = df.reset_index()

# Adding DataFrame Column Values

In [ ]:
df['col_total'] = df.col1 + df.col2

# Creating a Column Based on a Conditional

In [ ]:
data = {'name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'age': [42, 52, 36, 24, 73], 
        'preTestScore': [4, 24, 31, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70]}
df = pd.DataFrame(data, columns = ['name', 'age', 'preTestScore', 'postTestScore'])

df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

# Creating a Column Based on Existing Columns

In [ ]:
df = pd.DataFrame({'Date':['10/2/2011', '11/2/2011', '12/2/2011', '13/2/2011'], 
                    'Event':['Music', 'Poetry', 'Theatre', 'Comedy'], 
                    'Cost':[10000, 5000, 15000, 2000]})

df['Discounted_Price'] = df.apply(lambda row: row.Cost - 
                                  (row.Cost * 0.1), axis = 1)

# Selecting Non-null Values

In [ ]:
df = df.loc[df['column_name'].notnull()]

# Selecting Rows Where a Column is Null

In [ ]:
df = df[df['col_name'].isnull()]

# Selecting Rows Where Column is in List of Values

In [ ]:
df = df[df['col_name'].isin(list_of_values)]

# Selecting Rows Where Column is Not in List of Values

In [ ]:
df = df[~df['col_name'].isin(list_of_values)]

# Selecting Rows Based on Multiple Conditions

In [ ]:
df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]

# Value Counts

In [ ]:
df.col_name.value_counts()

# Value Counts Table

In [ ]:
labels = pd.concat([df.rating.value_counts(),
                    df.rating.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

# Making a List from Value Counts Index

In [ ]:
list_of_index_values = df.col_name.value_counts(dropna=False).index.tolist()

# Making a DataFrame from Value Counts

In [ ]:
df.column_name.value_counts().to_frame()

# Info and Describe

In [ ]:
df.info()
df.describe

# Working with Timestamps

In [ ]:
from datetime import datetime
from dateutil.parser import parse

df.timestamp[:1]

dtz = []
for ts in df.timestamp:
  dtz.append(parse(ts))
dtz[:10]

df['date_time_zone'] = df.apply(lambda row: parse(row.timestamp), axis=1)

df.set_index('date_time_zone', inplace=True)

# Designating CSAT vs DSAT

In [ ]:
# creates a new column and designates a row as either high or low
df['csat'] = np.where(df['rating']>=3, 'high', 'low')

# Splitting CSAT and DSAT

In [ ]:
df_positive =  df.loc[df['column_name'] == 'positive']
df_negative =  df.loc[df['column_name'] == 'negative']

In [ ]:
df.head()

# Correlation Heatmaps

In [ ]:
# Use the default Pearson for continuous variables
corr_matrix = df_totals.corr(method ='pearson')

# Use Spearman for ordinal variables
corr_matrix = df_totals.corr(method ='spearman')

# Setup
fig, ax = plt.subplots(figsize=(8,6))

# vmin and vmax control the range of the colormap
sns.heatmap(corr_matrix, cmap='RdBu', annot=True, fmt='.2f',
           vmin=-1, vmax=1)

plt.title("Correlations Between Something and Somethings")

# Add tight_layout to ensure the labels don't get cut off
plt.tight_layout()
plt.show()

# Pairplots

In [ ]:
g = sns.pairplot(df)

# Violin Plots

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.violinplot(x="category", y="numeric", data=df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

# Catplots

In [ ]:
g = sns.catplot(x="category1", y="numeric_measure",
                hue="binary_category", col="category2",
                data=df, kind="violin", split=True,
                height=6, aspect=1);

# Levene's Test

In [ ]:
stats.levene(df_group1['col_name'], df_group2['col_name'])

# Levene's Test Loop

In [ ]:
levene_hom = []
levene_het = []

for column in columns_list:
    
    result = stats.levene(df_group1[column], df_group2[column])[1]
    
    if result > ALPHA:
        interpretation = 'insignificant - HOMOGENOUS'
        levene_hom.append(column)
    else:
        interpretation = 'significant - HETEROGENOUS'
        levene_het.append(column)
        
    print(result, '-', column, ' - ', interpretation)

# Shapiro Test

In [ ]:
diff = df_group1['col_name'] - df_group2['col_name']
stats.shapiro(diff)

# Mann-Whitney U Test

In [ ]:
ALPHA = 0.05

stat, p = mannwhitneyu(df_group1[column], df_group2[column], df_group3[column])
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

# Mann-Whitney U Test Loop

In [ ]:
ALPHA = 0.05

mannwhitneyu_same = []
mannwhitneyu_diff = []

for column in columns_list:
    stat, p = mannwhitneyu(df_group1[column], df_group2[column], df_group3[column])
    
    if p > ALPHA:
        interpretation = 'SAME (fail to reject H0)'
        print('Statistics=%.3f, p=%.3f' % (stat, p) + ' - ' + column + ' - ' + interpretation)
        mannwhitneyu_same.append(column)
    else:
        interpretation = 'DIFFERENT (reject H0)'
        print('Statistics=%.3f, p=%.3f' % (stat, p) + ' - ' + column + ' - ' + interpretation)
        mannwhitneyu_diff.append(column)

# Independent T-testing

In [ ]:
stats.ttest_ind(df_group1['col_name'], df_group2['col_name'])

# Independent T-testing Loop

In [ ]:
ALPHA = 0.05

for column in levene_hom:
    
    result = stats.ttest_ind(df_group1[column], df_group2[column])[1]
    
    if result > ALPHA:
        interpretation = 'insignificant - SAME'
        ttest_same.append(column)
    else:
        interpretation = 'significant - DIFFERENT'
        ttest_diff.append(column)
        
    print(result, '-', column, ' - ', interpretation)

# Hierarchical Treemap

In [ ]:
fig = px.treemap(df, path=['case_origin', 'case_tag'], values='score')
fig.show()